### Requirements For the Project

In [1]:
# !pip install langchain langgraph langchain-google-genai requests pydantic pandas ipython jupyter python-dotenv fastapi python-multipart pyngrok nest_asyncio uvicorn

### Part 1: Transport Query Agent with FastAPI

#### 1. Imports and Setup

In [2]:
import os
import requests
from typing import List, Optional, Dict, Any, TypedDict, Annotated
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, ToolMessage
from dotenv import load_dotenv
import uvicorn
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from langgraph.checkpoint.memory import MemorySaver
import uuid
import time

load_dotenv()

True

#### 2. Pydantic Models for API Data Structures
#
#### We define Pydantic models to ensure that the data returned by our tools is structured, validated, and easy for the LLM to understand.


In [3]:
class BusArrival(BaseModel):
    """Data model for bus arrival times."""
    service_no: str = Field(..., description="The bus service number.")
    next_bus_arrival: str = Field(..., description="Estimated arrival time of the next bus.")
    next_bus_load: str = Field(..., description="Passenger load of the next bus (e.g., 'Seats Available').")
    bus_type: str = Field(..., description="The type of bus (e.g., 'SD' for single deck).")

class TaxiAvailability(BaseModel):
    """Data model for taxi availability."""
    latitude: float = Field(..., description="The latitude of the available taxi.")
    longitude: float = Field(..., description="The longitude of the available taxi.")

class TrafficIncident(BaseModel):
    """Data model for traffic incidents."""
    type: str = Field(..., description="The type of traffic incident (e.g., 'Road Work', 'Accident').")
    message: str = Field(..., description="A detailed message describing the incident.")
    latitude: float = Field(..., description="The latitude of the incident.")
    longitude: float = Field(..., description="The longitude of the incident.")

#### 3. Tool Definitions
#
#### These are the functions the agent can call to interact with the LTA DataMall API.


In [4]:
@tool
def get_bus_arrival_times(bus_stop_code: str, service_no: Optional[str] = None) -> List[BusArrival]:
    """
    Fetches real-time bus arrival times for a specific bus stop code.
    """
    api_key = os.getenv("LTA_DATAMALL_API_KEY")
    if not api_key: return "LTA DataMall API key not found."

    url = "https://datamall2.mytransport.sg/ltaodataservice/v3/BusArrival"
    headers = {'AccountKey': api_key}
    params = {'BusStopCode': bus_stop_code}

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        arrivals = []
        for s in data.get('Services', []):
            if service_no and s.get('ServiceNo') != service_no:
                continue

            for bus_key in ['NextBus', 'NextBus2', 'NextBus3']:
                bus_info = s.get(bus_key, {})
                if bus_info and bus_info.get('EstimatedArrival'):
                    arrivals.append(
                        BusArrival(
                            service_no=s.get('ServiceNo', 'N/A'),
                            next_bus_arrival=bus_info.get('EstimatedArrival', 'N/A'),
                            next_bus_load=bus_info.get('Load', 'N/A'),
                            bus_type=bus_info.get('Type', 'N/A'),
                        )
                    )
        return arrivals
    except Exception as e:
        return f"An error occurred: {e}"

@tool
def get_taxi_availability() -> List[TaxiAvailability]:
    """
    Fetches the current locations of all available taxis across Singapore.
    """
    api_key = os.getenv("LTA_DATAMALL_API_KEY")
    if not api_key: return "LTA DataMall API key not found."

    url = "https://datamall2.mytransport.sg/ltaodataservice/Taxi-Availability"
    headers = {'AccountKey': api_key}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()

        return [
            TaxiAvailability(latitude=t['Latitude'], longitude=t['Longitude'])
            for t in data.get('value', [])
        ]
    except Exception as e:
        return f"An error occurred: {e}"

@tool
def get_traffic_incidents() -> List[TrafficIncident]:
    """
    Fetches real-time traffic incidents, such as accidents or road work.
    """
    api_key = os.getenv("LTA_DATAMALL_API_KEY")
    if not api_key: return "LTA DataMall API key not found."

    url = "https://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents"
    headers = {'AccountKey': api_key}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()

        return [
            TrafficIncident(
                type=i.get('Type', 'N/A'),
                message=i.get('Message', 'N/A'),
                latitude=i.get('Latitude', 0.0),
                longitude=i.get('Longitude', 0.0)
            ) for i in data.get('value', [])
        ]
    except Exception as e:
        return f"An error occurred: {e}"

#### 4. Agent and Graph Definition
#
### Here, we define the state, nodes, and edges of our LangGraph workflow. The agent uses Gemini-2.5-flash and is equipped with the three tools defined above.


In [5]:
class AgentState(TypedDict):
    """Represents the state of our agent, including the history of messages."""
    messages: Annotated[list, add_messages]

# Initialize the language model and the list of tools
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
tools = [get_bus_arrival_times, get_taxi_availability, get_traffic_incidents]
llm_with_tools = llm.bind_tools(tools)

def agent_node(state: AgentState) -> Dict[str, Any]:
    print("Agent Node State:", state)
    time.sleep(2) 
    result = llm_with_tools.invoke(state["messages"])
    print("LLM Result:", result)
    return {"messages": [result]}

def tool_node(state: AgentState) -> Dict[str, Any]:
    print("Tool Node State:", state)
    tool_calls = state["messages"][-1].tool_calls
    print("Tool Calls:", tool_calls)
    tool_messages = []
    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        tool_to_call = next(filter(lambda t: t.name == tool_name, tools), None)
        if tool_to_call:
            observation = tool_to_call.invoke(tool_call["args"])
            print(f"Tool {tool_name} Result:", observation)
            tool_messages.append(
                ToolMessage(content=str(observation), tool_call_id=tool_call["id"])
            )
    return {"messages": tool_messages}

def should_continue(state: AgentState) -> str:
    """Determines the next step: call tools or end."""
    if state["messages"][-1].tool_calls:
        return "tools"
    return END

# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)
workflow.set_entry_point("agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

# Compile the graph into a runnable application
memory = MemorySaver()
app_graph = workflow.compile(checkpointer=memory)

#### 5. FastAPI Application
#
#### We wrap our LangGraph agent in a FastAPI server to expose its functionality as a REST API.


In [6]:
app_fastapi = FastAPI(
    title="Transport Query Agent API",
    description="An API for querying Singapore public transport data using a LangGraph agent.",
    version="1.0.0",
)

class QueryRequest(BaseModel):
    """Request model for a user query."""
    query: str

import logging
logging.basicConfig(level=logging.DEBUG)

@app_fastapi.post("/query")
async def query_agent(request: QueryRequest):
    try:
        thread_id = str(uuid.uuid4())
        config = {"configurable": {"thread_id": thread_id}}

        initial_state = {"messages": [HumanMessage(content=request.query)]}

        final_state = app_graph.invoke(initial_state, config=config)

        response_message = final_state["messages"][-1].content
        return JSONResponse(content={"response": response_message})
    except Exception as e:
        logging.error(f"Error processing query: {str(e)}", exc_info=True)
        return JSONResponse(status_code=500, content={"error": str(e)})

#### 6. Main Execution Block
#
#### This block runs the FastAPI server using Uvicorn. The server will be accessible at `http://127.0.0.1:8000`.


In [ ]:
if __name__ == "__main__":
    import nest_asyncio
    import uvicorn

    nest_asyncio.apply()

    print("Starting FastAPI server locally...")
    print("Access the API at http://127.0.0.1:8000")
    print("API documentation available at http://127.0.0.1:8000/docs")

    uvicorn.run(app_fastapi, host="127.0.0.1", port=8000)


Starting FastAPI server locally...
Access the API at http://127.0.0.1:8000
API documentation available at http://127.0.0.1:8000/docs


INFO:     Started server process [21228]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Agent Node State: {'messages': [HumanMessage(content='When is the next bus for service 169 arriving at bus stop 58241?', additional_kwargs={}, response_metadata={}, id='7b7c532c-7b9e-420b-b7bb-36b3067176b9')]}


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): datamall2.mytransport.sg:443


LLM Result: content='' additional_kwargs={'function_call': {'name': 'get_bus_arrival_times', 'arguments': '{"bus_stop_code": "58241", "service_no": "169"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--85760e55-d85e-49f9-ad95-266d62dfdc4f-0' tool_calls=[{'name': 'get_bus_arrival_times', 'args': {'bus_stop_code': '58241', 'service_no': '169'}, 'id': '7c985919-2a71-49d9-98a2-2276f54134de', 'type': 'tool_call'}] usage_metadata={'input_tokens': 175, 'output_tokens': 146, 'total_tokens': 321, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 110}}
Tool Node State: {'messages': [HumanMessage(content='When is the next bus for service 169 arriving at bus stop 58241?', additional_kwargs={}, response_metadata={}, id='7b7c532c-7b9e-420b-b7bb-36b3067176b9'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_bus_arrival_ti

...

Tool get_traffic_incidents Result: [TrafficIncident(type='Vehicle breakdown', message='(6/9)17:24 Vehicle Breakdown on AYE (towards Tuas) before Telok Blangah Rd.', latitude=1.272593617374595, longitude=103.84474725840384), TrafficIncident(type='Accident', message='(6/9)17:22 Accident in KPE Tunnel (towards ECP) at Pan Island Expressway Entrance.', latitude=1.3158886656492734, longitude=103.87496197338032), TrafficIncident(type='Vehicle breakdown', message='(6/9)16:46 Vehicle Breakdown on AYE (towards MCE) before Jurong Town Hall Rd. Avoid lane 4.', latitude=1.3267434297303002, longitude=103.733204177795), TrafficIncident(type='Roadwork', message='(6/9)15:49 Road Works on Xilin Avenue (towards ECP) before ECP. Avoid lanes 2 and 3.', latitude=1.3266688333636907, longitude=103.96845700104524), TrafficIncident(type='Roadwork', message='(6/9)15:31 Road Works on AYE (towards MCE) at Jurong Town Hall Road Entrance.', latitude=1.3229669615127884, longitude=103.74869532571118), TrafficIncident

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): datamall2.mytransport.sg:443


LLM Result: content='' additional_kwargs={'function_call': {'name': 'get_traffic_incidents', 'arguments': '{}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--2c8761f2-2eeb-40bf-ac27-06d8bc359e89-0' tool_calls=[{'name': 'get_traffic_incidents', 'args': {}, 'id': 'e1006a86-adc9-43ae-89d0-9ecf054970e4', 'type': 'tool_call'}] usage_metadata={'input_tokens': 161, 'output_tokens': 165, 'total_tokens': 326, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 152}}
Tool Node State: {'messages': [HumanMessage(content='Are there any accidents near Orchard Road?', additional_kwargs={}, response_metadata={}, id='c59567f4-dd92-403f-b4d2-d4fc5e43fd26'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_traffic_incidents', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'fin

DEBUG:urllib3.connectionpool:https://datamall2.mytransport.sg:443 "GET /ltaodataservice/TrafficIncidents HTTP/1.1" 200 None


Tool get_traffic_incidents Result: [TrafficIncident(type='Vehicle breakdown', message='(6/9)17:24 Vehicle Breakdown on AYE (towards Tuas) before Telok Blangah Rd.', latitude=1.272593617374595, longitude=103.84474725840384), TrafficIncident(type='Accident', message='(6/9)17:22 Accident in KPE Tunnel (towards ECP) at Pan Island Expressway Entrance.', latitude=1.3158886656492734, longitude=103.87496197338032), TrafficIncident(type='Vehicle breakdown', message='(6/9)16:46 Vehicle Breakdown on AYE (towards MCE) before Jurong Town Hall Rd. Avoid lane 4.', latitude=1.3267434297303002, longitude=103.733204177795), TrafficIncident(type='Roadwork', message='(6/9)15:49 Road Works on Xilin Avenue (towards ECP) before ECP. Avoid lanes 2 and 3.', latitude=1.3266688333636907, longitude=103.96845700104524), TrafficIncident(type='Roadwork', message='(6/9)15:31 Road Works on AYE (towards MCE) at Jurong Town Hall Road Entrance.', latitude=1.3229669615127884, longitude=103.74869532571118), TrafficIncident